In [ ]:
import numpy as np
import torch
from utils import LinCKA2
import matplotlib.pyplot as plt
from scipy.stats import special_ortho_group
import sys
from torch import nn

In [ ]:
def inv_lin_cka_cubes(mean1 = 0,
                var1 = 1,
                mean2 = 0,
                var2 = 1,
                transform_mean = 0,
                transform_var = 1,
                num_dims = 200,
                num_pts = 10000,
                seed = 0,
                distribution = 'gaussian'):
    
    np.random.seed(seed)
    rotation_matrix = np.random.normal(transform_mean, transform_var,[num_dims, num_dims])
    
    # Verify condition number until the matrix is
    while np.linalg.cond(rotation_matrix) >= 1/sys.float_info.epsilon:
        rotation_matrix = np.random.normal(transform_mean, transform_var,[num_dims, num_dims])

    cuda = torch.device('cuda')
    rotation_matrix = torch.Tensor(rotation_matrix).to(cuda)
    
    if distribution == 'gaussian':
        X = np.random.normal(mean1, var1, [int(num_pts*2), num_dims])
    elif distribution == 'uniform':
        # in this case var = side and mean = center
        X = np.concatenate([var1*(np.random.rand(num_pts, num_dims)-var1*0.5*np.ones([num_pts,num_dims]))+mean1*np.concatenate([np.ones([num_pts,1]),np.zeros([num_pts,num_dims-1])], axis=1), var2*(np.random.rand(num_pts, num_dims)-var2*0.5*np.ones([num_pts,num_dims]))+mean2*np.concatenate([np.ones([num_pts,1]),np.zeros([num_pts,num_dims-1])], axis=1)], axis = 0)

    X = torch.Tensor(X).to(cuda)
    Y = torch.mm(X,rotation_matrix)

    CKA = LinCKA2()
    return CKA(X,Y).item()

In [ ]:
X = np.concatenate([var1*(np.random.rand(num_pts, num_dims)-0.5*np.ones([num_pts,num_dims]))+mean1*np.concatenate([np.ones([num_pts,1]),np.zeros([num_pts,num_dims-1])], axis=1), var2*(np.random.rand(num_pts, num_dims)-0.5*np.ones([num_pts,num_dims]))+mean2*np.concatenate([np.ones([num_pts,1]),np.zeros([num_pts,num_dims-1])], axis=1)], axis = 0)


In [ ]:
num_pts = 10000#, 10000 and 20000 points makes it crash have other things running as well...
num_dims = 500
num_seeds = 10
mu_list = [1, 5, 10, 25, 50, 100, 500, 1000, 5000, 1e4, 1e5]
sigma_list = [1, 5, 10, 25, 50, 100, 500, 1000, 5000, 1e4, 1e5]


data = np.zeros([num_seeds, len(mu_list), len(sigma_list)])
for seed in range(num_seeds):
    print(f'seed {seed}')
    for i1, mu in enumerate(mu_list):
        for i2, sigma in enumerate(sigma_list):
            data[seed, i1, i2] = inv_lin_cka_cubes(mean1=0, mean2=1.1,num_dims = num_dims, num_pts = num_pts, transform_mean=mu, transform_var=sigma, seed = seed, distribution = 'uniform')
            
np.save('inv_lin_cka_fig_data_lincka2_10k_cubes_means0_1.1.npy', data)